# Chicago Tree Cover relative to Community Area

1. Read in tree canopy polygons and percentages to compare
2. Read in Chicago Community areas to overlay
3. Create new csv to use in clustering

---Tree Data Citation---<br>
link:<br> 
https://portal.edirepository.org/nis/mapbrowse?packageid=knb-lter-bes.5008.1

Citation:<br>
Locke, D.H. 2020. Residential housing segregation and urban tree canopy in 37 US Cities; <br>
data in support of Locke et al 2020 in npj Urban Sustainability ver 1. Environmental Data <br>Initiative.https://doi.org/10.6073/pasta/907a557cac781eb4cf72f866a2903c04 (Accessed 2021-03-01).

<hr>


In [1]:
#Libraries
import pandas as pd
import numpy as np
import geopandas as gdp
import requests
from bs4 import BeautifulSoup
import geocoder
from datetime import date
import json
from pandas.io.json import json_normalize
import folium
import geojson
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import re
from sklearn.cluster import KMeans
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [14]:
def parsedesc(descstr):
    lim = len(descstr)-1
    outstr=""
    for x in range(0, lim):
        if str(descstr[x-1]+descstr[x])=='1b':
            for y in range(x+2, lim):
                if descstr[y]=='"' or descstr[y]=="'":
                    for z in range(y+1, lim):
                        if descstr[z]!="'" and descstr[z]!='"':
                            outstr = outstr + descstr[z]
                        else:
                            return outstr

### 1.) Read in tree canopy polygons and percentages to compare
Data downloaded from citation source to local machine and read using geopandas
<hr>

In [2]:
test = gdp.read_file("C:/Users/Tony/Documents/Classes/Coursera/HOLC_Chicago/HOLC_Chicago_gdb.gdb")

In [3]:
geo = test.to_crs({'init': 'epsg:4326'})

C:\Users\Tony\anaconda3\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [4]:
test.head()

,name,holc_id,holc_grade,area_descr,dhl_id,Shape_Length,Shape_Area,TC_ID,TREE_CANOPY,Total_A,Can_P,geometry
0,,A1,A,"{ """" : """" }",0_A1_A,4546.464317,908012.353373,1,520571.0,883961.0,58.890720,"MULTIPOLYGON (((669252.281 2168904.149, 669128..."
1,,A10,A,"{ ""1a"" : "" Slowly "", ""1b"" : ""Upper class to be...",1_A10_A,4061.317231,509941.601357,2,301329.0,509933.0,59.091881,"MULTIPOLYGON (((676029.934 2155545.317, 676049..."
2,,A11,A,"{ ""1a"" : ""Slowly "", ""1b"" : ""Upper class"", ""1c...",2_A11_A,1854.006790,108148.613709,3,65853.0,108156.0,60.887052,"MULTIPOLYGON (((676869.385 2156664.478, 676688..."
3,,A12,A,"{ ""1a"" : "" Slowly "", ""1b"" : ""Upper class"", ""1c...",3_A12_A,3350.716595,298677.214210,4,169656.0,293609.0,57.782970,"MULTIPOLYGON (((677510.938 2155405.029, 677340..."
4,,A13,A,"{ ""1a"" : "" Steadily "", ""1b"" : ""Upper middle cl...",4_A13_A,3106.199671,236296.984057,5,143667.0,236293.0,60.800362,"MULTIPOLYGON (((676613.200 2155008.276, 676788..."


In [8]:
data = [geo['holc_id'], geo['Can_P']]
h = ['holc_id', 'Can_P']
df2 = pd.concat(data, axis=1, keys=h)
df2

,holc_id,Can_P
0,A1,58.890720
1,A10,59.091881
2,A11,60.887052
3,A12,57.782970
4,A13,60.800362
...,...,...
575,D95,19.577288
576,D96,15.159038
577,D97,22.577867
578,D98,24.138786


In [10]:
import folium

In [11]:
address = 'Chicago Illinois, USA'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Geograpical coordinates are {}, {}.'.format(latitude, longitude))

Geograpical coordinates are 41.8755616, -87.6244212.


In [12]:
latitude = 41.8900
longitude = -87.6393

In [13]:
f = folium.Figure(width=1000, height=1000)

m = folium.Map(location=[latitude, longitude], 
               zoom_start=10).add_to(f)

ch = folium.Choropleth(
                  geo_data=geo, 
                  data=df2,
                  columns=['holc_id', 'Can_P'], 
                  key_on=str('feature.properties.holc_id'),
                  fill_color='YlGn',
                  fill_opacity=0.7,
                  name="Tree Cover"
).add_to(m)

style_function = "font-size: 15px; font-weight: bold"
ch.geojson.add_child(
    folium.features.GeoJsonTooltip(['holc_id'], style=style_function, labels=False))
 
folium.LatLngPopup().add_to(m)
folium.LayerControl().add_to(m)

m

In [101]:
address = '6407 South Sacramento Avenue, Chicago Illinois, USA'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Geograpical coordinates are {}, {}.'.format(latitude, longitude))

Geograpical coordinates are 41.77692245, -87.69794132981791.


In [112]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [52]:
def findtreeply(pt, geotab, geocol):
    for x in range(0, len(geotab)-1):
        ply = geotab.iat[x, geocol]
        if pt.within(ply):
            lst = [geotab.iat[x, 1], geotab.iat[x, 10]]
            return lst
    return [0,0]

### 2.) Read in Chicago Community areas to overlay
Chicago Community Areas found in previous step, read in from saved csv
<hr>

In [70]:
chi = pd.read_csv('ChicagoCommunityAreas1.csv')
chi.head()

,Geography,Income,Lat,Long
0,Chatham,32222.0,41.74,-87.61167
1,Austin,31634.0,41.90,-87.76000
2,Beverly,98416.0,41.71,-87.68000
3,Morgan Park,60747.0,41.69,-87.67000
4,Woodlawn,26415.0,41.78,-87.60000


In [71]:
tlst=[]
for x in range(0, chi.shape[0]):
    lt = chi.iat[x,2]
    lg = chi.iat[x,3]
    p = Point(lg, lt)
    t = findtreeply(p,geo,11)
    tlst.append(t)


In [72]:
tply=[]
tcov=[]
for x in range(0, len(tlst)):
    tply.append(tlst[x][0])
    tcov.append(tlst[x][1])

In [51]:
f = folium.Figure(width=1000, height=1000)

m = folium.Map(location=[latitude, longitude], 
               zoom_start=10).add_to(f)

ch = folium.Choropleth(
                  geo_data=geo, 
                  data=df2,
                  columns=['holc_id', 'Can_P'], 
                  key_on=str('feature.properties.holc_id'),
                  fill_color='YlGn',
                  fill_opacity=0.7,
                  name="Tree Cover"
).add_to(m)

for neighborhood, lat, lng in zip(chi['Geography'], chi['Lat'], chi['Long']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(m)  


style_function = "font-size: 15px; font-weight: bold"
ch.geojson.add_child(
    folium.features.GeoJsonTooltip(['holc_id'], style=style_function, labels=False))
 

folium.LatLngPopup().add_to(m)
folium.LayerControl().add_to(m)

m

### 3.) Create new csv to use in clustering<hr>

In [73]:
chi['TreeCover']=tcov
chi['TreePolygon']=tply

In [76]:
chi.to_csv('ChicagoCommunityAreas2.csv', index=False)